# Parameterizing the LHCb RICH system using the pidgan's `GAN` algorithm

**Author:** [mbarbetti](https://github.com/mbarbetti)

**Date created:** 12/10/2023

**Last modified:** 12/10/2023

**Description:** This tutorial demonstrates how to parameterize the high-level response of the LHCb RICH system using a Generative Adversarial Network (GAN) [[1](https://arxiv.org/abs/1406.2661)]. The code is written using the [pidgan](https://github.com/mbarbetti/pidgan) package that relies on TensorFlow and Keras as backends.

## Introduction

### What is LHCb?

The [**LHCb experiment**](https://lhcb-outreach.web.cern.ch) has been originally designed to study rare decays of particles containing $b$ and $c$ quarks produced at the Large Hadron Collider (LHC). The LHCb detector, shown in the background of the following photo, is a single-arm forward spectrometer covering the pseudorapidity range of $2 < \eta < 5$. The detector includes:

- **Tracking system** - used for high-precision measurements of the momentum of charged particles and the position of the primary vertices
- **Particle Identification (PID) system** - used to distiguish different species of traversing particles (i.e., muons, pions, kaons, protons)

The LHCb PID system counts two ring-imaging Cherenkov ([RICH](https://lhcb-outreach.web.cern.ch/detector/rich-detectors/)) detectors whose response allows to separate different types of charged hadrons (e.g., pions, kaons, protons) using the [Cherenkov radiation](https://en.wikipedia.org/wiki/Cherenkov_radiation) of the traversing particles.

<div align="center">
  <img src="https://raw.githubusercontent.com/mbarbetti/pidgan-notebooks/main/.github/images/lhcb.jpeg" width="800"/>
</div>

### What are GANs?

Generative Adversarial Networks [[1](https://arxiv.org/abs/1406.2661)] are a powerful class of _generative models_ based on the simultaneous training of two neural networks:

*  **Discriminator network** ($D$) - trained by a classification task to separate the generator output from the reference dataset
* **Generator network** ($G$) - trained by a simulation task to reproduce the reference dataset trying to fake the discriminator

The goal is that $D$ optimally discriminates on the origin of the two samples, and simultaneously the training procedure for $G$ is to maximize the _probability_ of $D$ making a mistake. This framework corresponds to a **minimax two-player game** [[1](https://arxiv.org/abs/1406.2661)].

<div align="center">
  <img src="https://raw.githubusercontent.com/mbarbetti/pidgan-notebooks/main/.github/images/gan-scheme.png" width="800"/>
</div>

#### Mathematical details

The generator $G(z)$, fed by elements $z$ sampled according to a known distribution $p_z$ (typically gaussian), maps the **latent space** $\mathcal{Z}$ to the reference dataset $\mathcal{X}$, inducing a distribution $p_\rm{gen}$ trained to match with the target distribution $p_\rm{ref}$. The discriminator $D(x)$ outputs a single scalar, readable as the **probability** that $x$ comes from the reference dataset rather than $G$. Hence, the optimization problem corresponds to train $D$ to maximize the probability of correct labelling, and simultaneously training $G$ to minimize $\log(1 - D(G(z)))$.

Defining the **loss function** $\mathcal{L}_{\rm{GAN}}$ as follows

<center>$\mathcal{L}_{\rm{GAN}} (\theta_d, \theta_g) = \mathbb{E}_{x \sim p_\rm{ref}} \left[ \log{D_{\theta_d}(x)} \right] + \mathbb{E}_{z \sim p_\rm{z}} \left[ \log(1 - D_{\theta_d}(G_{\theta_g}(z))) \right]$</center>

the _minimax game_ can be written in this form:

<center>$\displaystyle{\min_G \, \max_D \, \mathcal{L}_{\rm{GAN}} (\theta_d, \theta_g)}$</center>

A unique solution exists, with $G$ recovering the reference distribution $p_\rm{ref}$ and $D$ equal to 1/2 everywhere [[1](https://arxiv.org/abs/1406.2661)].

Traditional GAN systems suffer from many issues, particularly during the training phase:

* the generator _may collapse_ producing only a single sample or a small family of very similar samples (**mode collapse**)
* the two players _may oscillate_ during training rather than converging to the [**Nash equilibrium**](https://en.wikipedia.org/wiki/Nash_equilibrium)
* if _imbalance_ between the two players occurs, then the system is incapable of learning at all

All these drawbacks result from the [**vanishing gradient problem**](https://en.wikipedia.org/wiki/Vanishing_gradient_problem), namely the lack of information for the update of the $G$ parameters. This is due to the saturation of the $D$ that is so good in distinguishing the origin of the two samples that no errors remain to the $G$ to improve the generated space. To fix the problem, one can add _continuous noise_ to both $D$ and $G$. This trick allows to learn thanks to a non-zero gradient [[2](https://arxiv.org/abs/1701.04862)].

#### Using input conditions

Feeding the generator with additional information besides the latent space sample allows to **conditionate** its output. In particular, it's sufficient to concatenate the conditional features $x$ to the random noise $z$ passed as input to $G$ to make it able to take into account this information [[3](https://arxiv.org/abs/1411.1784)]: $y_{\rm{gen}}(x) = G(x, z)$. Obviously, to preserve the capability to learn through the minimax game, also the discriminator must be fed by the additional features $x$, simply concatenating them to either elements of the reference sample or of the generated space [[3](https://arxiv.org/abs/1411.1784)]: $D(x, y)$ with $y \in \{y_{\rm{ref}}, y_{\rm{gen}}\}$.

## Tutorial

The aim of this tutorial is to build a parameterization of the LHCb RICH detectors. Since we expect that the high-level response of the RICH system only depends on the kinematics of the traversing particles and the detector occupancy, the former can be parameterized training a **GAN model** properly conditioned [[3](https://arxiv.org/abs/1411.1784), [4](https://arxiv.org/abs/1905.11825)]. The pidgan package provides several GAN [`algorithms`](https://github.com/mbarbetti/pidgan/tree/main/src/pidgan/algorithms) implementing some of the most used tricks presented in the literature to stablize their training [[1](https://arxiv.org/abs/1406.2661), [2](https://arxiv.org/abs/1701.04862), [5](https://arxiv.org/abs/1606.03498)]. The pidgan's [`players`](https://github.com/mbarbetti/pidgan/tree/main/src/pidgan/players), namely the generator and discriminator network, are designed to be trained taking conditions as input. In this notebook we will see how to build a model for the RICH system using the pidgan's [`GAN`](https://github.com/mbarbetti/pidgan/blob/main/src/pidgan/algorithms/GAN.py) algorithm.

### Install

The first step is to install the [pidgan](https://github.com/mbarbetti/pidgan) package and some other ones that are typically needed in machine learning applications to High Energy Physics (i.e., uproot, scipy, scikit-learn, matplotlib).

In [2]:
%%capture
!pip install pidgan[lamarr]

Now, let's verify the correct installation of pidgan printing its version:

In [3]:
import pidgan

pidgan.__version__

'0.0.5'

In case you're running this notebook on a machine equipped with a GPU, let's also verify the correct installation of TensorFlow:

In [4]:
import tensorflow as tf

tf.config.list_physical_devices("GPU")  # outputs a non-empty list in case of GPU equipped

[]

Finally, we just have to import all the modules that we will use in the following code cells:

In [5]:
import yaml
import uproot

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.utils import shuffle

### Data loading

### Data preprocessing

### Model definition

### Training procedure

### Validation plots

### Model export

## References

1. I.J. Goodfellow _et al._, "Generative Adversarial Networks", [arXiv:1406.2661](https://arxiv.org/abs/1406.2661)
2. M. Arjovsky, L. Bottou, "Towards Principled Methods for Training Generative Adversarial Networks", [arXiv:1701.04862](https://arxiv.org/abs/1701.04862)
3. M. Mirza, S. Osindero, "Conditional Generative Adversarial Nets", [arXiv:1411.1784](https://arxiv.org/abs/1411.1784)
4. A. Maevskiy _et al._, "Fast Data-Driven Simulation of Cherenkov Detectors Using Generative Adversarial Networks", [arXiv:1905.11825](https://arxiv.org/abs/1905.11825)
5. T. Salimans _et al._, "Improved Techniques for Training GANs", [arXiv:1606.03498](https://arxiv.org/abs/1606.03498)

## Credits
This tutorial is based on the notebooks provided by the [mbarbetti/tf-gen-models](https://github.com/mbarbetti/tf-gen-models) repository.